In [1]:
import pandas as pd
import os
import numpy as np
from pathlib import Path
import math
import torch
from copy import deepcopy
from itertools import chain 
from torch.utils.data import Dataset
from torchtext.vocab import vocab as Vocab

import warnings
warnings.filterwarnings("ignore")


Pathing


In [2]:
#Lokalt
data_dir = 'c:\\Users\\erika\\Desktop\\Exjobb\\data'

In [2]:
#saga
data_dir = "/home/aeerik/data/raw/"


Data import

In [5]:
from data_preprocessing import data_loader
include_pheno = False
threshold_year = 1970
path = data_dir

NCBI = data_loader(include_pheno,threshold_year,path)
NCBI.head()

,year,location,genes
1,[PAD],USA,"[sul1, tet(A), aadA1]"
4,[PAD],USA,"[sul2, aph(3'')-Ib, tet(A), aph(6)-Id]"
5,[PAD],USA,"[glpT_E448K=POINT, pmrB_Y358N=POINT]"
6,[PAD],Sweden,"[uhpT_E350Q=POINT, cyaA_S352T=POINT, glpT_E448..."
7,1979,USA,[glpT_E448K=POINT]


Vocabulary

In [6]:
from build_vocabulary import make_vocabulary
vocabulary = make_vocabulary(NCBI)
print(len(vocabulary))

1208


In [3]:
from create_dataset import NCBIDataset
from build_vocabulary import make_vocabulary
from data_preprocessing import data_loader

include_pheno = False
threshold_year = 1970
path = data_dir

NCBI = data_loader(include_pheno,threshold_year,path)

max_length = 20
mask_prob = 0.30
vocabulary = make_vocabulary(NCBI)

test_set = NCBIDataset(NCBI, vocabulary, max_length, mask_prob)
test_set.prepare_dataset()

In [36]:
test_set[1][2]

tensor([[False,  True, False, False, False, False, False,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True]])

Embedding 

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class JointEmbedding(nn.Module):

    def __init__(self, embedding_dim, vocab_size, max_length, drop_prob):
        super(JointEmbedding, self).__init__()

        self.max_length = max_length
        self.drop_prob = drop_prob
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        self.token_emb = nn.Embedding(self.vocab_size, self.embedding_dim)

        self.dropout = nn.Dropout(drop_prob)	
        self.norm = nn.LayerNorm(self.embedding_dim)

    def forward(self, input_tensor):
        token_embedding = self.token_emb(input_tensor)
        token_embedding = self.norm(token_embedding)
        token_embedding = self.dropout(token_embedding)

        return token_embedding


In [4]:
from embedding import JointEmbedding
embedding_dim = 32
voca_size = len(vocabulary)
max_length = 20
drop_prob = 0.2

emb_test = JointEmbedding(embedding_dim, voca_size, max_length, drop_prob)
emb_test.forward(test_set[1][0])

tensor([[-0.0000, -0.0000, -0.0000,  0.3553, -1.1371,  0.3805,  1.6536, -1.9498,
          0.1008,  0.9090,  1.2419, -1.7474,  1.6039,  2.3396, -0.1568,  0.0000,
          2.4389, -0.3773, -1.4833,  0.0000, -0.0000,  0.2353, -0.0151,  0.0000,
         -2.0182,  0.9652,  0.2159, -0.2871,  0.1594,  0.3802,  0.0000, -0.8101],
        [-1.2321,  1.8984,  0.7912,  2.1833, -0.8161,  0.0000, -0.4657, -0.6055,
          2.2380,  1.0590,  0.1244,  1.4381, -0.6233,  0.5699, -1.3460, -0.9220,
         -0.3099, -0.2273,  1.8147, -0.0000,  1.1527, -0.7644,  0.2706, -0.2831,
         -0.3776, -1.7533, -0.7721, -1.4761, -0.0000, -1.4792, -1.6105,  2.2412],
        [-0.0000, -0.5733,  2.6986, -0.6970,  0.4722, -0.6928,  1.1357, -1.1755,
         -0.8313,  0.2363, -0.4323,  0.3252, -3.1352, -2.6159, -0.6505, -0.0469,
         -0.2779,  0.6093,  0.0724, -0.6377,  1.7117,  1.2104,  0.4811,  0.0000,
          0.7456, -1.2218,  1.3354,  0.0000, -0.0000,  1.9296, -0.5335,  0.2506],
        [ 0.0483, -2.1863

In [16]:
test_set[0][0]

tensor([  0,   1,  59, 213, 214, 215,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1])

In [9]:
import torch

from torch import nn
import torch.nn.functional as f

class AttentionHead(nn.Module):

    def __init__(self, dim_inp, dim_out, drop_prob):
        super(AttentionHead, self).__init__()

        self.dim_inp = dim_inp
        self.drop_prob = drop_prob
        self.dropout = nn.Dropout(drop_prob)
        self.q = nn.Linear(dim_inp, dim_out)
        self.k = nn.Linear(dim_inp, dim_out)
        self.v = nn.Linear(dim_inp, dim_out)

    def forward(self, input_tensor: torch.Tensor, attention_mask: torch.Tensor = None):
        query, key, value = self.q(input_tensor), self.k(input_tensor), self.v(input_tensor)

        scale = query.size(1) ** 0.5
        scores = torch.matmul(query, key.transpose(-1, -2)) / scale

        scores = scores.masked_fill_(attention_mask, -1e9)
        attn = f.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        context = torch.matmul(attn, value)

        return context


class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, dim_inp, dim_out,drop_prob):
        super(MultiHeadAttention, self).__init__()

        self.heads = nn.ModuleList([
            AttentionHead(dim_inp, dim_out,drop_prob) for _ in range(num_heads)
        ])
        self.linear = nn.Linear(dim_out * num_heads, dim_inp)
        self.norm = nn.LayerNorm(dim_inp)

    def forward(self, input_tensor: torch.Tensor, attention_mask: torch.Tensor):
        s = [head(input_tensor, attention_mask) for head in self.heads]
        scores = torch.cat(s, dim=-1)
        scores = self.linear(scores)
        return self.norm(scores)
    
attention_test = AttentionHead(32, 32, 0.2)
attention_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

multihead_test = MultiHeadAttention(8, 32, 32,0.2)
multihead_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

tensor([[-6.8843e-01, -1.3420e+00,  2.9944e-01, -4.1503e-01, -1.6473e+00,
         -9.0050e-01, -2.0309e+00,  2.2498e-01, -1.7748e-01, -1.4642e+00,
          1.2188e+00,  1.7456e-01, -2.1747e+00,  1.4280e+00,  4.9190e-01,
          4.7069e-01, -3.4982e-01, -1.4850e-01, -6.1501e-01, -9.6792e-02,
          1.4723e+00,  1.0816e+00, -2.4056e-03,  3.5799e-01,  4.8171e-01,
          3.6956e-01,  1.4174e-01, -1.3759e-01,  1.5727e+00, -3.5759e-01,
          1.4613e+00,  1.3010e+00],
        [ 6.8182e-02, -1.2260e+00,  8.5642e-03, -5.8040e-01, -1.4073e+00,
         -7.4120e-01, -1.6831e+00, -5.5534e-01, -1.5944e+00, -1.3240e+00,
          4.4706e-01,  1.5174e-01, -1.2163e+00,  7.2857e-01,  5.9617e-01,
          9.6149e-01, -1.0766e-01,  7.0815e-01, -1.5241e+00, -8.5263e-01,
          2.4098e+00,  1.4359e+00,  6.4953e-02,  7.3426e-01,  3.8040e-01,
          5.1044e-01,  9.2362e-02, -5.4984e-02,  1.4878e+00,  9.2974e-02,
          7.9555e-01,  1.1931e+00],
        [-5.3420e-01, -3.1101e-01,  9.14

In [5]:
from bert_builder import AttentionHead

attention_test = AttentionHead(32, 32, 0.2)
attention_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

tensor([[ 2.7308e-02,  1.2452e-01, -6.6884e-03,  5.5833e-02,  1.2116e-01,
         -1.9990e-02,  8.4978e-01, -1.6624e-01, -3.9880e-01,  2.6900e-01,
          2.3491e-01, -6.2202e-02, -6.1828e-01,  4.7934e-01,  2.3262e-02,
         -3.9426e-01,  8.2099e-01, -2.4035e-01, -5.6881e-02, -8.0375e-01,
          7.5708e-01, -5.1566e-01, -2.3887e-01, -1.1806e+00,  6.1366e-01,
          5.7547e-01,  2.9847e-01, -2.2838e-01, -7.8742e-01, -3.8740e-01,
          5.2382e-01, -7.7262e-01],
        [ 9.2858e-02, -7.6474e-02,  8.5543e-02,  5.2141e-02,  2.9463e-01,
         -1.1200e-01,  3.4645e-01,  1.0705e-01,  1.1619e-01,  1.1212e-01,
          5.6340e-01,  2.1974e-01, -5.4030e-01,  6.6198e-01, -1.4842e-02,
         -6.3676e-02,  5.5643e-01, -2.7908e-01, -1.3047e-01, -6.1584e-01,
          3.9355e-01, -2.7492e-01, -2.3767e-01, -5.9836e-01,  4.4019e-01,
          4.3393e-01,  2.5858e-01,  5.8096e-02, -3.7804e-01, -1.3082e-01,
          2.9361e-01, -5.0309e-01],
        [-2.4001e-02,  2.0574e-01, -5.16

---------------------------

dataset
- behöver vocabulary
- saknar: getitem och prepare dataset som kallar på construct_masking

In [1]:
from torchtext.vocab import vocab 
from copy import deepcopy
from collections import Counter
import torch
from torch.utils.data import Dataset
import pandas as pd
from itertools import chain
import numpy as np

from data_preprocessing import data_loader

include_pheno = True
threshold_year = 1970

NCBI = data_loader(include_pheno,threshold_year)
NCBI.fillna('[PAD]', inplace=True)
print('data importerad')


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MASK_PERCENTAGE = 0.15


# data, vocabulary, max sequence length, mask probability, include sequences, some random state
class NCBIDataset(Dataset):

    MASKED_INDICES_COLUMN = 'masked_indices'
    TARGET_COLUMN = 'indices'
    NSP_TARGET_COLUMN = 'is_next'
    TOKEN_MASK_COLUMN = 'token_mask'

    def __init__(self,
                 data: pd.DataFrame,
                 vocab: vocab,
                 max_seq_len: int,
                 mask_prob: float,
                 random_state: int = 23,
                 ):
        
        self.random_state = random_state
        np.random.seed(self.random_state)

        CLS = '[CLS]'
        PAD = '[PAD]'
        MASK = '[MASK]'
        UNK = '[UNK]'

        self.data = data.reset_index(drop=True) 
        self.num_samples = self.data.shape[0]
        self.vocab = vocab
        self.vocab_size = len(self.vocab)
        self.CLS = CLS 
        self.PAD = PAD
        self.MASK = MASK
        self.UNK = UNK
        self.max_seq_len = max_seq_len
        self.mask_prob = mask_prob
        self.columns = [self.MASKED_INDICES_COLUMN, self.TARGET_COLUMN]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        input = torch.Tensor(item[self.MASKED_INDICES_COLUMN],device=device).long()
        token_mask  = torch.tensor(item[self.TARGET_COLUMN], device=device).long()
        attention_mask = (input == self.vocab[self.PAD]).unsqueeze(0)

        return input, token_mask , attention_mask

    def _construct_masking(self):
        sequences = deepcopy(self.data['genes'].tolist())
        masked_sequences = list()
        target_indices_list = list()
        seq_starts = [[self.CLS, self.data['year'].iloc[i], self.data['location'].iloc[i]] for i in range(self.data.shape[0])]

        for i, geno_seq in enumerate(sequences):
            seq_len = len(geno_seq)
            masking_index = np.random.rand(seq_len) < self.mask_prob   
            target_indices = np.array([-1]*seq_len)
            indices = masking_index.nonzero()[0]
            target_indices[indices] = self.vocab.lookup_indices([geno_seq[i] for i in indices])
            for i in indices:
                r = np.random.rand()
                if r < 0.8:
                    geno_seq[i] = self.MASK
                elif r > 0.9:
                    geno_seq[i] = self.vocab.lookup_token(np.random.randint(self.vocab_size))
            geno_seq = seq_starts[i] + geno_seq
            target_indices = [-1]*3 + target_indices.tolist() 
            masked_sequences.append(geno_seq)
            target_indices_list.append(target_indices)
        print('här är lugnt')
        print(masked_sequences[:20])
        print(target_indices_list[:20])
        masked_sequences = [seq + [self.PAD]*(self.max_seq_len - len(seq)) for seq in masked_sequences]
        print(masked_sequences[:20])
        print(range(len(target_indices_list)))
        for i in range(len(target_indices_list)):
            indices = target_indices_list[i]
            padding = [-1] * (self.max_seq_len - len(indices))
            target_indices_list[i] = indices + padding
        print('hit men inte längre ')
        return masked_sequences, target_indices_list 
        
    def prepare_dataset(self):
        masked_sequences, target_indices = self._construct_masking()
        indices_masked = [self.vocab.lookup_indices(masked_seq) for masked_seq in masked_sequences]

        rows = zip(indices_masked, target_indices)
        self.data = pd.DataFrame(rows, columns=self.columns)
        print(self.data.head())

def make_vocabulary(dataset: pd.DataFrame):
    CLS = '[CLS]'
    PAD = '[PAD]'
    MASK = '[MASK]'
    UNK = '[UNK]'

    token_list = Counter()
    data = dataset.copy()

    location_tokens = list(dict.fromkeys(list(chain(list(data['location'])))))
    year_tokens = list(dict.fromkeys(list(chain(list(data['year'])))))
    genes_tokens = list(dict.fromkeys(list(chain(*data['genes']))))
   
    token_list.update(map(str, year_tokens))
    token_list.update(map(str, location_tokens))
    token_list.update(map(str, genes_tokens))
    vocabulary = vocab(token_list,specials = [CLS, PAD, MASK, UNK])
    return vocabulary


max_length = 10
mask_prob = 0.30
vocabulary = make_vocabulary(NCBI)

test_set = NCBIDataset(NCBI, vocabulary, max_length, mask_prob)
test_set.prepare_dataset()

data importerad
här är lugnt
[['[CLS]', '[PAD]', 'USA', 'aadA1', 'sul1', 'tet(A)'], ['[CLS]', '[PAD]', 'Sweden', '[MASK]', '[MASK]', 'tet(A)', '[MASK]'], ['[CLS]', '[PAD]', 'USA', 'glpT_E448K=POINT', 'pmrB_Y358N=POINT'], ['[CLS]', '[PAD]', 'Sweden', 'glpT_E448K=POINT', 'cyaA_S352T=POINT', 'uhpT_E350Q=POINT'], ['[CLS]', '1979', 'USA', 'glpT_E448K=POINT'], ['[CLS]', '[PAD]', 'USA', 'glpT_E448K=POINT', '[MASK]', '[MASK]'], ['[CLS]', '[PAD]', 'USA', '[MASK]', 'pmrB_E123D=POINT', '[MASK]'], ['[CLS]', '[PAD]', 'USA', 'glpT_E448K=POINT', 'pmrB_Y358N=POINT'], ['[CLS]', '[PAD]', 'USA', 'glpT_E448K=POINT', 'pmrB_Y358N=POINT'], ['[CLS]', '[PAD]', 'Sweden', 'parC_A56T=POINT', 'glpT_E448K=POINT', '[MASK]', '[MASK]'], ['[CLS]', '[PAD]', 'Sweden', 'parE_I355T=POINT'], ['[CLS]', '[PAD]', 'Sweden', 'glpT_E448K=POINT'], ['[CLS]', '[PAD]', 'Sweden', "aph(3'')-Ib", 'tet(B)', 'sul2', '[MASK]'], ['[CLS]', '[PAD]', 'Indonesia', 'glpT_E448K=POINT'], ['[CLS]', '[PAD]', 'USA', 'glpT_E448K=POINT', 'pmrB_E123D=PO

In [2]:
from torchtext.vocab import vocab 
from copy import deepcopy
from collections import Counter
import torch
from torch.utils.data import Dataset
import pandas as pd
from itertools import chain
import numpy as np

from data_preprocessing import data_loader
from create_dataset import NCBIDataset

include_pheno = True
threshold_year = 1970

NCBI = data_loader(include_pheno,threshold_year)
print('data importerad')

def make_vocabulary(dataset: pd.DataFrame):
    CLS = '[CLS]'
    PAD = '[PAD]'
    MASK = '[MASK]'
    UNK = '[UNK]'

    token_list = Counter()
    data = dataset.copy()

    location_tokens = list(dict.fromkeys(list(chain(list(data['location'])))))
    year_tokens = list(dict.fromkeys(list(chain(list(data['year'])))))
    genes_tokens = list(dict.fromkeys(list(chain(*data['genes']))))
   
    token_list.update(map(str, year_tokens))
    token_list.update(map(str, location_tokens))
    token_list.update(map(str, genes_tokens))
    vocabulary = vocab(token_list,specials = [CLS, PAD, MASK, UNK])
    return vocabulary


max_length = 10
mask_prob = 0.30
vocabulary = make_vocabulary(NCBI)

test_set = NCBIDataset(NCBI, vocabulary, max_length, mask_prob)
test_set.prepare_dataset()

data importerad
här är lugnt
[['[CLS]', nan, 'USA', 'sul1', 'tet(A)', 'aadA1'], ['[CLS]', nan, 'Sweden', '[MASK]', '[MASK]', 'aph(6)-Id', '[MASK]'], ['[CLS]', nan, 'USA', 'pmrB_Y358N=POINT', 'glpT_E448K=POINT'], ['[CLS]', nan, 'Sweden', 'uhpT_E350Q=POINT', 'cyaA_S352T=POINT', 'glpT_E448K=POINT'], ['[CLS]', '1979', 'USA', 'glpT_E448K=POINT'], ['[CLS]', nan, 'USA', 'pmrB_E123D=POINT', '[MASK]', '[MASK]'], ['[CLS]', nan, 'USA', '[MASK]', 'pmrB_E123D=POINT', '[MASK]'], ['[CLS]', nan, 'USA', 'pmrB_Y358N=POINT', 'glpT_E448K=POINT'], ['[CLS]', nan, 'USA', 'pmrB_Y358N=POINT', 'glpT_E448K=POINT'], ['[CLS]', nan, 'Sweden', 'pmrB_Y358N=POINT', 'tet(A)', '[MASK]', '[MASK]'], ['[CLS]', nan, 'Sweden', 'parE_I355T=POINT'], ['[CLS]', nan, 'Sweden', 'glpT_E448K=POINT'], ['[CLS]', nan, 'Sweden', 'sul2', "aph(3'')-Ib", 'aph(6)-Id', '[MASK]'], ['[CLS]', nan, 'Indonesia', 'glpT_E448K=POINT'], ['[CLS]', nan, 'USA', 'pmrB_E123D=POINT', 'glpT_E448K=POINT'], ['[CLS]', nan, 'USA', 'aph(6)-Id', 'glpT_E448K=POINT

: 

Vocabulary

In [ ]:
import random
import typing
from collections import Counter
from pathlib import Path

import numpy as np
import pandas as pd
from itertools import chain
import torch

from tqdm import tqdm
from torch.utils.data import Dataset
from torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer 

print(list(chain(*NCBI['genes']))[:20])

mylist = list(dict.fromkeys(list(chain(*NCBI['genes']))))
print(len(mylist))
print(mylist[:20])

test = list(chain(list(NCBI['year'])))
print(test[:40])
test = list(dict.fromkeys(list(chain(list(NCBI['year'])))))
print(len(test))
print(test[:40])

test = list(chain(list(NCBI['location'])))
print(test[:20])
test = list(dict.fromkeys(list(chain(list(NCBI['location'])))))
print(len(test))
print(test[:20])

In [ ]:
import random
import typing
from collections import Counter
from pathlib import Path

import numpy as np
import pandas as pd
from itertools import chain
import torch

from tqdm import tqdm
from torch.utils.data import Dataset
from torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer 

def make_vocabulary(dataset: pd.DataFrame):
    CLS = '[CLS]'
    PAD = '[PAD]'
    SEP = '[SEP]'
    MASK = '[MASK]'
    UNK = '[UNK]'

    token_list = Counter()
    data = dataset.copy()

    location_tokens = list(dict.fromkeys(list(chain(list(data['location'])))))
    year_tokens = list(dict.fromkeys(list(chain(list(data['year'])))))
    genes_tokens = list(dict.fromkeys(list(chain(*data['genes']))))
   
    token_list.update(map(str, year_tokens))
    token_list.update(map(str, location_tokens))
    token_list.update(map(str, genes_tokens))
    print(token_list)
    vocabulary = vocab(token_list,specials = [CLS, PAD, MASK, SEP, UNK])
    print(vocabulary)
    return vocabulary
vocabulary = make_vocabulary(NCBI)


Counter({'nan': 2, '1979': 1, '2013': 1, '1996': 1, '1975': 1, '2009': 1, '1974': 1, '1998': 1, '1983': 1, '2008': 1, '1995': 1, '1989': 1, '2003': 1, '1997': 1, '1994': 1, '2001': 1, '1980': 1, '1970': 1, '1971': 1, '2007': 1, '2004': 1, '2006': 1, '1982': 1, '1991': 1, '1990': 1, '2002': 1, '1988': 1, '1987': 1, '2011': 1, '1986': 1, '1985': 1, '2010': 1, '2005': 1, '2012': 1, '1984': 1, '2000': 1, '2014': 1, '2015': 1, '1999': 1, '1992': 1, '1973': 1, '1981': 1, '1993': 1, '2016': 1, '1972': 1, '1976': 1, '1977': 1, '2017': 1, '1978': 1, '2018': 1, '2019': 1, '2020': 1, '2021': 1, '2022': 1, '2023': 1, '2024': 1, 'USA': 1, 'Sweden': 1, 'Indonesia': 1, 'Canada': 1, 'Papua New Guinea': 1, 'Japan': 1, 'China': 1, 'Germany': 1, 'India': 1, 'Bangladesh': 1, 'Brazil': 1, 'UK': 1, 'Denmark': 1, 'South Korea': 1, 'France': 1, 'Norway': 1, 'Malaysia': 1, 'Slovenia': 1, 'Thailand': 1, 'Italy': 1, 'Israel': 1, 'Austria': 1, 'Belgium': 1, 'Guinea-Bissau': 1, 'Australia': 1, 'Hungary': 1, 'Leban

In [ ]:
vocabulary = make_vocabulary(NCBI)
